# LangChain: Memory

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

#replace the OpenAI API key with AzureOpenAI API key
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai_deployment_id = os.getenv("OPENAI_DEPLOYMENT_ID")

In [3]:
from langchain.chat_models import AzureChatOpenAI # replace with AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [4]:
# llm = ChatOpenAI(temperature=0.0)
llm=AzureChatOpenAI(deployment_name=openai_deployment_id, model_name=openai_deployment_id, temperature=0)

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [5]:
conversation.predict(input="Hi, my name is Andrew")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [7]:
conversation.predict(input="What is my name?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

In [8]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?\nHuman: What is 1+1?\nAI: The answer to 1+1 is 2.\nHuman: What is my name?\nAI: Your name is Andrew, as you mentioned earlier."}

In [10]:
memory = ConversationBufferMemory()

In [11]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [14]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [16]:
from langchain.memory import ConversationBufferWindowMemory

In [17]:
memory = ConversationBufferWindowMemory(k=1)               

In [18]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [19]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [20]:
# llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [21]:
conversation.predict(input="Hi, my name is Andrew")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [22]:
conversation.predict(input="What is 1+1?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [23]:
memory.load_memory_variables({})

{'history': 'Human: What is 1+1?\nAI: The answer to 1+1 is 2.'}

In [24]:
conversation.predict(input="What is my name?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory

In [36]:
# !pip install tiktoken

In [96]:
# OpenAI embeddings
response = openai.Embedding.create(
    input="hello world",
    deployment_id="text-embedding-ada-002"
    # deployment_id="text-davinci-003"
)
embeddings = response['data'][0]['embedding']
len(embeddings)

1536

In [69]:
response.keys()

dict_keys(['object', 'data', 'model', 'usage'])

In [70]:
response.data[0].keys()

dict_keys(['object', 'index', 'embedding'])

In [71]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import AzureOpenAI # replace with AzureOpenAI
# llm = ChatOpenAI(temperature=0.0)
llm=AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

In [72]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [73]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [75]:
from langchain.memory import ConversationSummaryBufferMemory


In [76]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

llm=AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [77]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool".\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [79]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [80]:
conversation.predict(input="What would be a good demo to show?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool".
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


' A good demo to show would be the latest version of the LLM demo. It showcases the latest features and capabilities of the AI, and it is sure to impress the customer.'

In [81]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop.\nHuman: What would be a good demo to show?\nAI:  A good demo to show would be the latest version of the LLM demo. It showcases the latest features and capabilities of the AI, and it is sure to impress the customer.'}

In [82]:
conversation.predict(input="prepare a demo please.")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop.
Human: What would be a good demo to show?
AI:  A good demo to show would be the latest version of the LLM demo. It showcases the latest features and capabilities of the AI, and it is sure to impress the customer.
Human: prepare a demo please.
AI:

> Finished chain.


" Sure, I can prepare a demo for you. I'll need some information from you first, such as what features you would like to highlight and what type of environment the demo will be presented in. Once I have that information, I can create a customized demo that will be sure to impress the customer."

In [91]:
print(memory.load_memory_variables({})['history'])

System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop. The AI suggests the latest version of the LLM demo as a good demonstration to show, as it showcases the latest features and capabilities of the AI.
Human: prepare a demo please.
AI:  Sure, I can prepare a demo for you. I'll need some information from you first, such as what features you would like to highlight and what type of environment the demo will be presented in. Once I have that information, I can create a customized demo that will be sure to impress the customer.


In [92]:
conversation.predict(input="what is your suggestion?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop. The AI suggests the latest version of the LLM demo as a good demonstration to show, as it showcases the latest features and capabilities of the AI.
Human: prepare a demo please.
AI:  Sure, I can prepare a demo for you. I'll need some information from you first, such as what features you would like to highlight and wha

' My suggestion would be to focus on the latest features and capabilities of the AI, as they are the most impressive and will likely be the most interesting to the customer. Additionally, you should consider the environment the demo will be presented in, as this will help determine which features to focus on.'

In [93]:
print(memory.load_memory_variables({})['history'])

System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop. The AI suggests the latest version of the LLM demo as a good demonstration to show, as it showcases the latest features and capabilities of the AI. When asked to prepare a demo, the AI offers to create a customized demo based on the features the human would like to highlight and the environment the demo will be presented in.
Human: what is your suggestion?
AI:  My suggestion would be to focus on the latest features and capabilities of the AI, as they are the most impressive and will likely be the most interesting to the customer. Additionally, you should consider the environment the demo wil

In [94]:
conversation.predict(input="any other suggestions?")

Error in on_chain_start callback: 'name'


Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop. The AI suggests the latest version of the LLM demo as a good demonstration to show, as it showcases the latest features and capabilities of the AI. When asked to prepare a demo, the AI offers to create a customized demo based on the features the human would like to highlight and the environment the demo will be presen

' Yes, I suggest customizing the demo to highlight the features that are most relevant to the customer and the environment they will be in. This will ensure that the demo is tailored to their needs and will be more engaging.'

In [95]:
print(memory.load_memory_variables({})['history'])

System: 
The human greets the AI and asks what is on the schedule for the day. The AI responds with a casual "Cool" and informs the human that there is a meeting at 8am with the product team, followed by time to work on the LangChain project from 9am-12pm. At noon, the human is expected to have lunch with a customer at an Italian restaurant to demonstrate the latest LLM demo, bringing their laptop. The AI suggests the latest version of the LLM demo as a good demonstration to show, as it showcases the latest features and capabilities of the AI. When asked for a suggestion, the AI recommends focusing on the latest features and capabilities of the AI, as they are the most impressive and will likely be the most interesting to the customer, and considering the environment the demo will be presented in, as this will help determine which features to focus on.
Human: any other suggestions?
AI:  Yes, I suggest customizing the demo to highlight the features that are most relevant to the customer